# Ход работы Rgr

## Подключение библиотек

In [1]:
import pandas as pd #обработка (загрузка, сохранение, анализа) данных
import numpy  as np #обработка многомерных массивов, линейная алгебра, преобразование Фурье, случайные числа

from sklearn import ( metrics, linear_model )
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline
import tensorflow as tf
import os

import json
import pickle

## Объявление функций

In [2]:
def GET_METRICS(y_test, y_pred):
    '''
    Вычисление и вывод метрик: MAE, RMSE, R2.
    На основе сравнения проверочных и вычисленных.
    :param y_test: - проверочные значения целевой переменный
    :param y_pred: - вычисленные значения целевой переменный
    '''
    mae  = metrics.mean_absolute_error        (y_test, y_pred)
    mse  = metrics.mean_squared_error         (y_test, y_pred)
    rmse = np.sqrt                            (mse)
    r2   = metrics.r2_score                   (y_test, y_pred)

    #print ("MAE : {:>9,.3f} (средняя абсолютная ошибка)".format              ( mae ))
    #print ("MSE : {:>9,.6f} (среднеквадратичная ошибка)".format( mse ))
    #print ("RMSE: {:>9,.6f} (кв. корень из среднеквадратичной ошибки)".format( rmse ))
    #print ("R2  : {:>9,.3f} (коэфф. детерминации)".format                    ( r2 ))
    return(mae,mse,rmse,r2)

## Выполнить загрузку файла с подготовленным набором данных (очищенный от пропусков и выбросов для целевой и независимых переменных) в таблицу исходных данных df

In [3]:
DataFrame=pd.read_csv('../insurance_data_Ready.csv',sep=',',header=0)

## Объявить список названий целевых (target) и независимой (features) переменных 

In [4]:
target=['charges']
features=['age','bmi','children','sex_female','sex_male','smoker_no','smoker_yes','reg_northeast','reg_northwest','reg_southeast','reg_southwest']

### Отделить X от Y исходной таблицы и сформировать новые таблицы dfX и dfY

In [5]:
DataFrame_X=DataFrame[features]
DataFrame_Y=DataFrame[target]

In [6]:
DataFrame_X

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,reg_northeast,reg_northwest,reg_southeast,reg_southwest
0,19.0,27.900,0.0,1,0,0,1,0,0,0,1
1,28.0,33.000,3.0,0,1,1,0,0,0,1,0
2,33.0,22.705,0.0,0,1,1,0,0,1,0,0
3,32.0,28.880,0.0,0,1,1,0,0,1,0,0
4,31.0,25.740,0.0,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1185,52.0,44.700,3.0,1,0,1,0,0,0,0,1
1186,50.0,30.970,3.0,0,1,1,0,0,1,0,0
1187,18.0,31.920,0.0,1,0,1,0,1,0,0,0
1188,21.0,25.800,0.0,1,0,1,0,0,0,0,1


In [7]:
DataFrame_Y

,charges
0,16884.92400
1,4449.46200
2,21984.47061
3,3866.85520
4,3756.62160
...,...
1185,11411.68500
1186,10600.54830
1187,2205.98080
1188,2007.94500


## Выполнить процедуру НОРМАЛИЗАЦИИ данных и сохранить в отдельную таблиц

In [8]:
scalerNorm_X = MinMaxScaler()
scalerNorm_X.fit(DataFrame_X)
DataFrame_X_Norm = pd.DataFrame (
  data    = scalerNorm_X.transform(DataFrame_X), 
  columns = DataFrame_X.columns,           
  index   = DataFrame_X.index              
)

scalerNorm_Y = MinMaxScaler()
scalerNorm_Y.fit(DataFrame_Y)
DataFrame_Y_Norm = pd.DataFrame (
  data    = scalerNorm_Y.transform(DataFrame_Y), 
  columns = DataFrame_Y.columns,           
  index   = DataFrame_Y.index              
)

## Сформировать обучающую и проверочную части

### На основе исходных данных dfX и dfY сформировать xTrain , xTest , yTrain , yTest

In [9]:
x_train, x_test, y_train, y_test =  train_test_split(
    DataFrame_X,    # исходные данные X
    DataFrame_Y,      # исходные данные y
    test_size = 0.2,
    random_state=115,
    shuffle=True          
) 

In [1]:
x_train

NameError: name 'x_train' is not defined

### На основе нормализованной выборки  dfXNorm и dfYNorm сформировать xNormTrain , xNormTest , yNormTrain , yNormTest

In [10]:
x_norm_train, x_norm_test, y_norm_train, y_norm_test =  train_test_split(
    DataFrame_X_Norm,    
    DataFrame_Y_Norm,      
    test_size = 0.2, 
    random_state=115, 
    shuffle=True          
) 

## Модель m1. Линейная регрессия

### Для построения линейной регрессионной модели выбрать из dfX группу  независимых признаков 'x1, x2, x3, ...'

In [11]:
target=['charges']
features=['age','bmi','children','sex_female','sex_male','smoker_no','smoker_yes','reg_northeast','reg_northwest','reg_southeast','reg_southwest']

### Создать и обучить на выбранных признаках модель LinearRegression() из библиотеки sklearn, используя xTrain, yTrain 

In [12]:
linear_model = linear_model.LinearRegression()
linear_model = linear_model.fit(x_train, y_train)

### Получить вычисленные значения yPred с использованием обученной модели на  данных xTest 

In [13]:
y_lin_pred = linear_model.predict(x_test)

### Получить оценки точности R2, RMSE для построенной модели, используя данные  yPred и yTest

In [14]:

mae_m1,mse_m1,rmse_m1,r2_m1 = GET_METRICS(y_test, y_lin_pred)

print ("MAE : {:>9,.3f} (средняя абсолютная ошибка)".format              ( mae_m1 ))
print ("MSE : {:>9,.6f} (среднеквадратичная ошибка)".format              ( mse_m1 ))
print ("RMSE: {:>9,.6f} (кв. корень из среднеквадратичной ошибки)".format( rmse_m1 ))
print ("R2  : {:>9,.3f} (коэфф. детерминации)".format                    ( r2_m1 ))


MAE : 4,135.144 (средняя абсолютная ошибка)
MSE : 31,436,191.547444 (среднеквадратичная ошибка)
RMSE: 5,606.798690 (кв. корень из среднеквадратичной ошибки)
R2  :     0.798 (коэфф. детерминации)


## Модель m2. Нейронная сеть

###	Выбрать из dfNormX группу  независимых признаков 'x1, x2, x3, ...'

In [15]:
target=['charges']
features=['age','bmi','children','sex_female','sex_male','smoker_no','smoker_yes','reg_northeast','reg_northwest','reg_southeast','reg_southwest']

###	Определить
-	Структуру одной моделей (кол-во узлов на входных, выходных и скрытых слоях и узлов на каждом из слоев).
-	Вид нелинейных функции активации для скрытых слоев
-	Функцию оценки потерь (способ оценки ошибок между yNormPred и yTrain) 
-	Функцию оптимизации
-   Создать модель, используя библиотеку tensorflow

In [16]:
with tf.device('/CPU:0'):
    totalHistoryLossTrain=[] # Вспомогательный список для хранение полной истории обучения
    totalHistoryLossTest=[]  # Вспомогательный список для хранение полной истории обучения
    # 1) Параметры структуры
    input_size  = 11  # кол-во входных  узлов
    output_size = 1  # кол-во выходных узлов
    # 2.1) Построение модели нейронной сети
    # Многослойная    
    neuro_model = tf.keras.models.Sequential()
    neuro_model.add(tf.keras.layers.Input(shape=(11,))) # Входной слой
    neuro_model.add(tf.keras.layers.Dense(units=110,  activation=tf.keras.activations.relu,  )) # Скрытй слой, полносвязный, с нелинейной функцией активации
    neuro_model.add(tf.keras.layers.Dense(units=55,  activation=tf.keras.activations.tanh  ))  # Скрытй слой, полносвязный, с нелинейной функцией активации
    neuro_model.add(tf.keras.layers.Dense(units=20,  activation=tf.keras.activations.sigmoid,  ))  # Скрытй слой, полносвязный, с нелинейной функцией активации
    neuro_model.add(tf.keras.layers.Dense(units=1,  activation=tf.keras.activations.linear)) # Выходной слой, с линейной функцией активации
    # 2.2) Ф. потерь и оптимизации
    fLoss     =tf.keras.losses.mean_squared_error
    fOptimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
    fMetric   =[tf.keras.losses.mean_squared_error]

    neuro_model.compile(
        loss      =fLoss, 
        optimizer =fOptimizer, 
        metrics   =[ fMetric ]
    )

### Обучить модель на выбранных признаках, используя нормализованные данные xNormTrain, yNormTrain

In [17]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

with tf.device('/CPU:0'):
    # 3) Цикл обучения/продолжения обучения сети             
    history = neuro_model.fit(
        x_norm_train, y_norm_train,

        validation_data=(x_norm_test, y_norm_test), 

        epochs=1000,   # кол-во эпох обучения
        batch_size=100, # кол-во образцов в каждой эпохе
        verbose=1,
    )

Epoch 1/1000
10/10 [==============================] - 3s 86ms/step - loss: 0.1030 - mean_squared_error: 0.1030 - val_loss: 0.0444 - val_mean_squared_error: 0.0444
Epoch 2/1000
10/10 [==============================] - 0s 27ms/step - loss: 0.0334 - mean_squared_error: 0.0334 - val_loss: 0.0185 - val_mean_squared_error: 0.0185
Epoch 3/1000
10/10 [==============================] - 0s 20ms/step - loss: 0.0219 - mean_squared_error: 0.0219 - val_loss: 0.0208 - val_mean_squared_error: 0.0208
Epoch 4/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0183 - mean_squared_error: 0.0183 - val_loss: 0.0159 - val_mean_squared_error: 0.0159
Epoch 5/1000
10/10 [==============================] - 0s 17ms/step - loss: 0.0165 - mean_squared_error: 0.0165 - val_loss: 0.0129 - val_mean_squared_error: 0.0129
Epoch 6/1000
10/10 [==============================] - 0s 51ms/step - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.0124 - val_mean_squared_error: 0.0124
Epoch 7/1000
10/10 [==

10/10 [==============================] - 0s 22ms/step - loss: 0.0087 - mean_squared_error: 0.0087 - val_loss: 0.0082 - val_mean_squared_error: 0.0082
Epoch 52/1000
10/10 [==============================] - 0s 14ms/step - loss: 0.0089 - mean_squared_error: 0.0089 - val_loss: 0.0082 - val_mean_squared_error: 0.0082
Epoch 53/1000
10/10 [==============================] - 0s 14ms/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.0085 - val_mean_squared_error: 0.0085
Epoch 54/1000
10/10 [==============================] - 0s 14ms/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.0085 - val_mean_squared_error: 0.0085
Epoch 55/1000
10/10 [==============================] - 0s 14ms/step - loss: 0.0087 - mean_squared_error: 0.0087 - val_loss: 0.0082 - val_mean_squared_error: 0.0082
Epoch 56/1000
10/10 [==============================] - 0s 14ms/step - loss: 0.0089 - mean_squared_error: 0.0089 - val_loss: 0.0082 - val_mean_squared_error: 0.0082
Epoch 57/1000
10/10 [=========

10/10 [==============================] - 0s 23ms/step - loss: 0.0080 - mean_squared_error: 0.0080 - val_loss: 0.0086 - val_mean_squared_error: 0.0086
Epoch 102/1000
10/10 [==============================] - 0s 22ms/step - loss: 0.0083 - mean_squared_error: 0.0083 - val_loss: 0.0081 - val_mean_squared_error: 0.0081
Epoch 103/1000
10/10 [==============================] - 0s 18ms/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.0087 - val_mean_squared_error: 0.0087
Epoch 104/1000
10/10 [==============================] - 0s 14ms/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.0087 - val_mean_squared_error: 0.0087
Epoch 105/1000
10/10 [==============================] - 0s 14ms/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.0089 - val_mean_squared_error: 0.0089
Epoch 106/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0087 - mean_squared_error: 0.0087 - val_loss: 0.0082 - val_mean_squared_error: 0.0082
Epoch 107/1000
10/10 [===

Epoch 151/1000
10/10 [==============================] - 0s 17ms/step - loss: 0.0075 - mean_squared_error: 0.0075 - val_loss: 0.0100 - val_mean_squared_error: 0.0100
Epoch 152/1000
10/10 [==============================] - 0s 11ms/step - loss: 0.0077 - mean_squared_error: 0.0077 - val_loss: 0.0094 - val_mean_squared_error: 0.0094
Epoch 153/1000
10/10 [==============================] - 0s 22ms/step - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.0098 - val_mean_squared_error: 0.0098
Epoch 154/1000
10/10 [==============================] - 0s 17ms/step - loss: 0.0077 - mean_squared_error: 0.0077 - val_loss: 0.0095 - val_mean_squared_error: 0.0095
Epoch 155/1000
10/10 [==============================] - 0s 14ms/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.0103 - val_mean_squared_error: 0.0103
Epoch 156/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0079 - mean_squared_error: 0.0079 - val_loss: 0.0083 - val_mean_squared_error: 0.0083
Epoch 157/

Epoch 201/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0075 - mean_squared_error: 0.0075 - val_loss: 0.0087 - val_mean_squared_error: 0.0087
Epoch 202/1000
10/10 [==============================] - 0s 14ms/step - loss: 0.0069 - mean_squared_error: 0.0069 - val_loss: 0.0086 - val_mean_squared_error: 0.0086
Epoch 203/1000
10/10 [==============================] - 0s 17ms/step - loss: 0.0070 - mean_squared_error: 0.0070 - val_loss: 0.0088 - val_mean_squared_error: 0.0088
Epoch 204/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.0069 - mean_squared_error: 0.0069 - val_loss: 0.0088 - val_mean_squared_error: 0.0088
Epoch 205/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.0095 - val_mean_squared_error: 0.0095
Epoch 206/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.0075 - mean_squared_error: 0.0075 - val_loss: 0.0090 - val_mean_squared_error: 0.0090
Epoch 207/

...

Epoch 751/1000
10/10 [==============================] - 0s 17ms/step - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.0112 - val_mean_squared_error: 0.0112
Epoch 752/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.0035 - mean_squared_error: 0.0035 - val_loss: 0.0118 - val_mean_squared_error: 0.0118
Epoch 753/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.0115 - val_mean_squared_error: 0.0115
Epoch 754/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0039 - mean_squared_error: 0.0039 - val_loss: 0.0128 - val_mean_squared_error: 0.0128
Epoch 755/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0038 - mean_squared_error: 0.0038 - val_loss: 0.0104 - val_mean_squared_error: 0.0104
Epoch 756/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0038 - mean_squared_error: 0.0038 - val_loss: 0.0119 - val_mean_squared_error: 0.0119
Epoch 757/

Epoch 801/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.0032 - mean_squared_error: 0.0032 - val_loss: 0.0123 - val_mean_squared_error: 0.0123
Epoch 802/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.0031 - mean_squared_error: 0.0031 - val_loss: 0.0125 - val_mean_squared_error: 0.0125
Epoch 803/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0118 - val_mean_squared_error: 0.0118
Epoch 804/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0142 - val_mean_squared_error: 0.0142
Epoch 805/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0038 - mean_squared_error: 0.0038 - val_loss: 0.0115 - val_mean_squared_error: 0.0115
Epoch 806/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0037 - mean_squared_error: 0.0037 - val_loss: 0.0129 - val_mean_squared_error: 0.0129
Epoch 807/

Epoch 851/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0030 - mean_squared_error: 0.0030 - val_loss: 0.0129 - val_mean_squared_error: 0.0129
Epoch 852/1000
10/10 [==============================] - 0s 11ms/step - loss: 0.0028 - mean_squared_error: 0.0028 - val_loss: 0.0125 - val_mean_squared_error: 0.0125
Epoch 853/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0029 - mean_squared_error: 0.0029 - val_loss: 0.0116 - val_mean_squared_error: 0.0116
Epoch 854/1000
10/10 [==============================] - 0s 14ms/step - loss: 0.0030 - mean_squared_error: 0.0030 - val_loss: 0.0121 - val_mean_squared_error: 0.0121
Epoch 855/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.0106 - val_mean_squared_error: 0.0106
Epoch 856/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0032 - mean_squared_error: 0.0032 - val_loss: 0.0135 - val_mean_squared_error: 0.0135
Epoch 857/

Epoch 901/1000
10/10 [==============================] - 0s 14ms/step - loss: 0.0027 - mean_squared_error: 0.0027 - val_loss: 0.0118 - val_mean_squared_error: 0.0118
Epoch 902/1000
10/10 [==============================] - 0s 14ms/step - loss: 0.0027 - mean_squared_error: 0.0027 - val_loss: 0.0127 - val_mean_squared_error: 0.0127
Epoch 903/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.0026 - mean_squared_error: 0.0026 - val_loss: 0.0116 - val_mean_squared_error: 0.0116
Epoch 904/1000
10/10 [==============================] - 0s 14ms/step - loss: 0.0028 - mean_squared_error: 0.0028 - val_loss: 0.0145 - val_mean_squared_error: 0.0145
Epoch 905/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0035 - mean_squared_error: 0.0035 - val_loss: 0.0130 - val_mean_squared_error: 0.0130
Epoch 906/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.0116 - val_mean_squared_error: 0.0116
Epoch 907/

Epoch 951/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0131 - val_mean_squared_error: 0.0131
Epoch 952/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.0027 - mean_squared_error: 0.0027 - val_loss: 0.0125 - val_mean_squared_error: 0.0125
Epoch 953/1000
10/10 [==============================] - 0s 14ms/step - loss: 0.0028 - mean_squared_error: 0.0028 - val_loss: 0.0121 - val_mean_squared_error: 0.0121
Epoch 954/1000
10/10 [==============================] - 0s 14ms/step - loss: 0.0028 - mean_squared_error: 0.0028 - val_loss: 0.0120 - val_mean_squared_error: 0.0120
Epoch 955/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.0027 - mean_squared_error: 0.0027 - val_loss: 0.0121 - val_mean_squared_error: 0.0121
Epoch 956/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.0026 - mean_squared_error: 0.0026 - val_loss: 0.0122 - val_mean_squared_error: 0.0122
Epoch 957/

### Получить вычисленные значения yNormPred с использованием обученной модели на  нормализованных данных xNormTest 

In [19]:
with tf.device('/CPU:0'):
    y_neuro_pred = neuro_model.predict( x_norm_test)

8/8 [==============================] - 0s 7ms/step


###	Получить оценки точности R2, RMSE для построенной модели, используя  нормализованные данные  yNormPred и yNormTest

In [20]:
y_test = scalerNorm_Y.inverse_transform(y_norm_test)
y_pred = scalerNorm_Y.inverse_transform(y_neuro_pred)


In [21]:

mae_m2,mse_m2,rmse_m2,r2_m2 = GET_METRICS(y_test, y_pred)

print ("MAE : {:>9,.3f} (средняя абсолютная ошибка)".format              ( mae_m2 ))
print ("MSE : {:>9,.6f} (среднеквадратичная ошибка)".format              ( mse_m2 ))
print ("RMSE: {:>9,.6f} (кв. корень из среднеквадратичной ошибки)".format( rmse_m2 ))
print ("R2  : {:>9,.3f} (коэфф. детерминации)".format                    ( r2_m2 ))

MAE : 3,210.750 (средняя абсолютная ошибка)
MSE : 31,373,825.779139 (среднеквадратичная ошибка)
RMSE: 5,601.234309 (кв. корень из среднеквадратичной ошибки)
R2  :     0.798 (коэфф. детерминации)


## Сохранение моделей. Для обученных моделей (m1, m2, …)



### Заполните следующие характеристики модели в структуре типа “ключ-значение”, где “ключ” название характеристики:
-	Тип модели
-	Требует нормализованные данные
-	Название признаков входных
-	Название признаков выходных
-	R2
-	RMSE
-	Выбранные вами дополнительные характеристики 


In [22]:
m1_dict=dict()
m1_dict['id']='0'
m1_dict['name']='Страхование от признаков'
m1_dict['ТипМодели']=str(type(linear_model))
m1_dict['НормализацияДанных']=False
m1_dict['СтандартизацияДанных']=False
m1_dict['НазваниеПризнаковВходных']=features
m1_dict['НазваниеПризнаковВыходных']=target
m1_dict['MAE']=mae_m1
m1_dict['MSE']=mse_m1
m1_dict['RMSE']=rmse_m1
m1_dict['R2']=r2_m1
m1_dict['dataset_name_forTrain']='insurance_data_Ready'
m1_dict['model_filename']='linear_model.h5'
m1_dict['y_test']='16884.924'

In [23]:

m2_dict=dict()
m2_dict['id']='1'
m2_dict['name']='Страхование от признаков'
m2_dict['ТипМодели']=str(type(neuro_model))
m2_dict['НормализацияДанных']=True
m2_dict['СтандартизацияДанных']=False
m2_dict['НазваниеПризнаковВходных']=features
m2_dict['НазваниеПризнаковВыходных']=target
m2_dict['MAE']=mae_m2
m2_dict['MSE']=mse_m2
m2_dict['RMSE']=rmse_m2
m2_dict['R2']=r2_m2
m2_dict['dataset_name_forTrain']='insurance_data_Ready'
m2_dict['model_filename']='neuro_model.h5'
m2_dict['y_test']='16884.924'

### Сохраните структуру типа “ключ-значение” в файле формата JSON, используя команду  json.dump(...)

In [24]:
with open('data_char_m1.json', 'w', encoding='utf-8') as f:
    json.dump(m1_dict, f, ensure_ascii=False, indent=4)
with open('data_char_m2.json', 'w', encoding='utf-8') as f:
    json.dump(m2_dict, f, ensure_ascii=False, indent=4)

### Для m2 сохраните объекты-шкалеры scalerNormForX и scalerNormForY с помощью механизма сериализации в файл, используя команду  pickle.dump(...)

In [25]:
with open('scalerNorm_X.pickle', 'wb') as handle:
    pickle.dump(scalerNorm_X, handle)
with open('scalerNorm_Y.pickle', 'wb') as handle:
    pickle.dump(scalerNorm_Y, handle)

##	Сохраните обученные модели с помощью механизма сериализации в файл:
-	Для моделей m1 используйте команду  pickle.dump(...)
-	Для моделей m2 используйте команду  tensorflow.save(...)


In [26]:
with open('linear_model_saved.pickle', 'wb') as handle:
    pickle.dump(linear_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

neuro_model.save(r"neuro_model_saved.h5", overwrite=True, save_format='h5')